In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('train.csv')

In [3]:
print(df.columns.tolist())

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

In [ ]:
#categorical columns
cols_to_cat=['MSSubClass', 'MSZoning','Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature','MoSold', 'YrSold', 'SaleType', 'SaleCondition']
for col in cols_to_cat:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.add_categories('Missing').fillna('Missing')

In [ ]:
#id is not a predictor
df = df.drop('Id', axis=1)
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,...,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,...,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,...,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,MnPrv,Missing,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,4,2010,WD,Normal,142125


In [ ]:
#cols types
df.dtypes

MSSubClass       category
MSZoning         category
LotFrontage       float64
LotArea             int64
Street           category
                   ...   
MoSold           category
YrSold           category
SaleType         category
SaleCondition    category
SalePrice           int64
Length: 80, dtype: object

In [ ]:
#check for NA
na_counts = df.isna().sum().sort_values(ascending=False)
print(na_counts)

LotFrontage     259
MasVnrArea        8
MSSubClass        0
KitchenAbvGr      0
GarageYrBlt       0
               ... 
ExterCond         0
ExterQual         0
MasVnrType        0
Exterior2nd       0
SalePrice         0
Length: 80, dtype: int64


In [ ]:
#fill NA with mean for LotFrontage
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())
df=df.dropna()

In [ ]:
#first ANOVA for categorical predictors
predictors = ['MSSubClass', 'MSZoning','Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature','MoSold', 'YrSold', 'SaleType', 'SaleCondition']

formula = 'SalePrice ~ ' + ' + '.join([f"C({col})" for col in predictors])
model = ols(formula, df).fit()

print(model.summary())


📋 Wybrane zmienne: ['OverallQual', 'Neighborhood', 'TotRmsAbvGrd', 'BsmtExposure', 'MSSubClass', 'RoofMatl', 'FullBath', 'Fireplaces', 'GarageCars', 'BsmtQual', 'Condition2']
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.895
Method:                 Least Squares   F-statistic:                     23.24
Date:                Mon, 14 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:32:06   Log-Likelihood:                -16451.
No. Observations:                1452   AIC:                         3.402e+04
Df Residuals:                     894   BIC:                         3.696e+04
Df Model:                         557                                         
Covariance Type:            nonrobust                                         
                                  

In [ ]:
#looking for best predictors using ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table_sorted = anova_table.sort_values(by='PR(>F)', ascending=True)
print(anova_table_sorted)

                       sum_sq     df          F        PR(>F)
C(RoofMatl)      9.128356e+10    8.0  17.314170  1.374694e-21
C(Neighborhood)  9.173090e+10   25.0   5.567686  8.029654e-16
C(FullBath)      5.598959e+10    4.0  21.239604  2.695333e-13
C(OverallQual)   5.901477e+10   10.0   8.954881  5.143046e-13
C(PoolQC)        3.742293e+10    3.0  18.928474  6.583797e-12
...                       ...    ...        ...           ...
C(MoSold)        3.733217e+09   12.0   0.472064  9.209012e-01
C(BsmtFinType2)  1.195278e+09    6.0   0.302285  9.358258e-01
C(PavedDrive)    1.444677e+07    3.0   0.007307  9.927195e-01
C(ExterCond)     1.483160e+08    5.0   0.045011  9.961757e-01
Residual         5.891670e+11  894.0        NaN           NaN

[61 rows x 4 columns]


In [66]:
# 2nd ANOVA for selected predictors
new_predictors = [
    'Neighborhood', 'BsmtExposure', 'HouseStyle','OverallQual', 'RoofMatl',
    'FullBath', 'GarageCars', 'TotRmsAbvGrd', 'MSSubClass', 'Condition2',
    'Fireplaces', 'BsmtQual',
]

formula = 'SalePrice ~ ' + ' + '.join([f"C({col})" for col in new_predictors])
model = ols(formula, df).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     94.49
Date:                Mon, 14 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:43:57   Log-Likelihood:                -16951.
No. Observations:                1452   AIC:                         3.410e+04
Df Residuals:                    1354   BIC:                         3.462e+04
Df Model:                          97                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [67]:
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table_sorted = anova_table.sort_values(by='PR(>F)', ascending=True)
print(anova_table_sorted)

                       sum_sq      df          F        PR(>F)
C(OverallQual)   2.996897e+11    10.0  34.564143  3.169788e-55
C(Neighborhood)  2.561209e+11    25.0  11.815683  2.121713e-41
C(RoofMatl)      1.017856e+11     8.0  14.674059  1.585369e-18
C(BsmtExposure)  7.876030e+10     4.0  22.709171  3.529007e-18
C(TotRmsAbvGrd)  1.025959e+11    12.0   9.860582  2.099609e-17
C(FullBath)      9.561786e+10     4.0  27.569758  2.632254e-17
C(Fireplaces)    9.078539e+10     4.0  26.176397  1.847794e-16
C(MSSubClass)    9.746521e+10    15.0   7.493976  2.451335e-15
C(GarageCars)    7.442599e+10     5.0  17.167558  9.585307e-14
C(BsmtQual)      5.536142e+10     4.0  15.962509  8.917844e-13
C(Condition2)    5.225346e+10     8.0   7.533191  6.316604e-09
C(HouseStyle)    1.042335e+10     8.0   1.502696  1.620867e-01
Residual         1.173991e+12  1354.0        NaN           NaN


In [68]:
#new dataframe with selected predictors
columns_to_keep = ['SalePrice'] + new_predictors
df_new = df[columns_to_keep].copy()
df_new.head()

,SalePrice,Neighborhood,BsmtExposure,HouseStyle,OverallQual,RoofMatl,FullBath,GarageCars,TotRmsAbvGrd,MSSubClass,Condition2,Fireplaces,BsmtQual
0,208500,CollgCr,No,2Story,7,CompShg,2,2,8,60,Norm,0,Gd
1,181500,Veenker,Gd,1Story,6,CompShg,2,2,6,20,Norm,1,Gd
2,223500,CollgCr,Mn,2Story,7,CompShg,2,2,6,60,Norm,1,Gd
3,140000,Crawfor,No,2Story,7,CompShg,1,3,7,70,Norm,1,TA
4,250000,NoRidge,Av,2Story,8,CompShg,2,3,9,60,Norm,1,Gd


In [69]:
#check for rare categories in categorical columns
cat_cols = df_new.select_dtypes(include='category').columns
category_counts = {}

for col in cat_cols:
    category_counts[col] = df_new[col].value_counts().to_dict()
print(category_counts)

{'Neighborhood': {'NAmes': 225, 'CollgCr': 149, 'OldTown': 113, 'Edwards': 100, 'Somerst': 83, 'Gilbert': 78, 'NridgHt': 76, 'Sawyer': 74, 'NWAmes': 73, 'BrkSide': 58, 'SawyerW': 58, 'Crawfor': 50, 'Mitchel': 49, 'NoRidge': 41, 'Timber': 38, 'IDOTRR': 37, 'ClearCr': 28, 'SWISU': 25, 'StoneBr': 25, 'MeadowV': 17, 'Blmngtn': 17, 'BrDale': 16, 'Veenker': 11, 'NPkVill': 9, 'Blueste': 2, 'Missing': 0}, 'BsmtExposure': {'No': 946, 'Av': 221, 'Gd': 133, 'Mn': 114, 'Missing': 38}, 'HouseStyle': {'1Story': 721, '2Story': 442, '1.5Fin': 154, 'SLvl': 65, 'SFoyer': 37, '1.5Unf': 14, '2.5Unf': 11, '2.5Fin': 8, 'Missing': 0}, 'OverallQual': {5: 397, 6: 372, 7: 315, 8: 167, 4: 116, 9: 43, 3: 20, 10: 17, 2: 3, 1: 2, 'Missing': 0}, 'RoofMatl': {'CompShg': 1426, 'Tar&Grv': 11, 'WdShngl': 6, 'WdShake': 5, 'ClyTile': 1, 'Membran': 1, 'Metal': 1, 'Roll': 1, 'Missing': 0}, 'FullBath': {2: 762, 1: 649, 3: 32, 0: 9, 'Missing': 0}, 'GarageCars': {2: 817, 1: 369, 3: 180, 0: 81, 4: 5, 'Missing': 0}, 'TotRmsAbvGr

In [70]:
# map for rare categories 
threshold = 30  


rare_map = {}
for col in cat_cols:
    counts = df_new[col].value_counts()
    rare = counts[counts < threshold].index.tolist()
    rare_map[col] = rare


for col in cat_cols:
    df_new[col] = df_new[col].apply(lambda x: 'Other' if x in rare_map[col] else x)

df_new.head()

,SalePrice,Neighborhood,BsmtExposure,HouseStyle,OverallQual,RoofMatl,FullBath,GarageCars,TotRmsAbvGrd,MSSubClass,Condition2,Fireplaces,BsmtQual
0,208500,CollgCr,No,2Story,7,CompShg,2,2,8,60,Norm,0,Gd
1,181500,Other,Gd,1Story,6,CompShg,2,2,6,20,Norm,1,Gd
2,223500,CollgCr,Mn,2Story,7,CompShg,2,2,6,60,Norm,1,Gd
3,140000,Crawfor,No,2Story,7,CompShg,1,3,7,70,Norm,1,TA
4,250000,NoRidge,Av,2Story,8,CompShg,2,3,9,60,Norm,1,Gd


In [71]:
# all numeric columns in df_new
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(num_cols)

['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']


In [72]:
# correlations in numeric variables with SalePrice
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
correlations = df[num_cols].corrwith(df['SalePrice']).sort_values(ascending=False)
print(correlations)

SalePrice        1.000000
GrLivArea        0.710080
GarageArea       0.622492
TotalBsmtSF      0.612971
1stFlrSF         0.606849
MasVnrArea       0.477493
BsmtFinSF1       0.383977
LotFrontage      0.333322
WoodDeckSF       0.324650
2ndFlrSF         0.322710
OpenPorchSF      0.311268
LotArea          0.264674
BsmtUnfSF        0.215740
ScreenPorch      0.113044
PoolArea         0.093109
3SsnPorch        0.045247
BsmtFinSF2      -0.010316
MiscVal         -0.020951
LowQualFinSF    -0.025263
EnclosedPorch   -0.128778
dtype: float64


In [73]:
# add extra numeric columns to df_new
extra_num_cols = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF','MasVnrArea']
for col in extra_num_cols:
    if col not in df_new.columns:
        df_new[col] = df[col]
df_new.head()

,SalePrice,Neighborhood,BsmtExposure,HouseStyle,OverallQual,RoofMatl,FullBath,GarageCars,TotRmsAbvGrd,MSSubClass,Condition2,Fireplaces,BsmtQual,GrLivArea,TotalBsmtSF,1stFlrSF,MasVnrArea
0,208500,CollgCr,No,2Story,7,CompShg,2,2,8,60,Norm,0,Gd,1710,856,856,196.0
1,181500,Other,Gd,1Story,6,CompShg,2,2,6,20,Norm,1,Gd,1262,1262,1262,0.0
2,223500,CollgCr,Mn,2Story,7,CompShg,2,2,6,60,Norm,1,Gd,1786,920,920,162.0
3,140000,Crawfor,No,2Story,7,CompShg,1,3,7,70,Norm,1,TA,1717,756,961,0.0
4,250000,NoRidge,Av,2Story,8,CompShg,2,3,9,60,Norm,1,Gd,2198,1145,1145,350.0


In [74]:
#data prep
X = df_new.drop('SalePrice', axis=1)
y = df_new['SalePrice']

for col in cat_cols:
    X[col] = X[col].astype(str)


cat_cols = X.select_dtypes(include='object').columns
num_cols = X.select_dtypes(include=[np.number]).columns


preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])


X_processed = preprocessor.fit_transform(X)

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#train/test split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
])


#compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

#training
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2, callbacks=[early_stop])

# accuracy
loss, mae = model.evaluate(X_test, y_test)
print(f"MAE: {mae}")
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R^2",round(r2,4))



Epoch 1/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 37167607808.0000 - mae: 178430.4531 - val_loss: 37080743936.0000 - val_mae: 175026.8438
Epoch 2/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 25897342976.0000 - mae: 139693.4219 - val_loss: 2863259648.0000 - val_mae: 42468.7031
Epoch 3/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 3803731456.0000 - mae: 37497.0508 - val_loss: 1456772992.0000 - val_mae: 27338.5059
Epoch 4/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2428424960.0000 - mae: 32135.5664 - val_loss: 1115970944.0000 - val_mae: 24453.7109
Epoch 5/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2400419072.0000 - mae: 29351.9980 - val_loss: 1034694528.0000 - val_mae: 23343.8535
Epoch 6/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1882389248.0000 - mae: 27219.2227 - val_loss: 962178368.0000 - val_mae: 22689.2305
Epoch 7/1000
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1389847808.0000 - mae: 25333.3301 - val_loss: 904360064.0000 - v